<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Data" data-toc-modified-id="Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data</a></span></li><li><span><a href="#Diagnostic-distributed" data-toc-modified-id="Diagnostic-distributed-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Diagnostic distributed</a></span><ul class="toc-item"><li><span><a href="#diagnostic-distributed-progressbar" data-toc-modified-id="diagnostic-distributed-progressbar-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>diagnostic distributed progressbar</a></span></li></ul></li></ul></div>

# Imports

In [4]:
import numpy as np
import pandas as pd

import dask
import dask.dataframe as dd
import dask.array as da
import dask_ml
import pyarrow

print([(x.__name__,x.__version__) for x in 
       [np,pd, dask, dask_ml,pyarrow]])

[('numpy', '1.18.5'), ('pandas', '1.0.5'), ('dask', '2.20.0'), ('dask_ml', '1.5.0'), ('pyarrow', '0.17.1')]


# Data

In [13]:
a = da.random.normal(size=(10_000, 10_000), chunks=(1000, 1000)) # data
res = a.dot(a.T).mean(axis=0) # operation


/Users/poudel/opt/miniconda3/envs/dsk/lib/python3.7/site-packages/dask/array/routines.py:272: PerformanceWarning: Increasing number of chunks by factor of 10
  axes=(left_axes, right_axes),


# Diagnostic distributed

- https://docs.dask.org/en/latest/diagnostics-distributed.html
- https://docs.dask.org/en/latest/diagnostics-local.html

It is typically served at http://localhost:8787/status , but may be served elsewhere if this port is taken. The address of the dashboard will be displayed if you are in a Jupyter Notebook, or can be queried from `client.scheduler_info()['services']`.

## diagnostic distributed progressbar

In [7]:
# from dask.diagnostics import ProgressBar # for single machine

# a = da.random.normal(size=(2000, 2000), chunks=(1000, 1000))
# res = a.dot(a.T).mean(axis=0)
# with ProgressBar():
#     out = res.compute()

In [8]:
# Distributed scheduler ProgressBar

from dask.distributed import Client, progress

client = Client(processes=False)  # use dask.distributed by default
client

/Users/poudel/.local/lib/python3.7/site-packages/distributed-2.9.3-py3.7.egg/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: inproc://192.168.0.188/1007/9 Dashboard: http://192.168.0.188/1007/9:49559/status,Cluster Workers: 1 Cores: 4 Memory: 17.18 GB


In [9]:
client.scheduler_info()['services']

{'dashboard': 49559}

In [15]:
res = res.persist()  # start computation in the background
progress(res)      # watch progress
res.compute()      # convert to final result when done if desired

distributed.scheduler - ERROR - Couldn't gather keys {"('mean_agg-aggregate-8a3aa5d5a7c11071fd31b41c9dfb8b0c', 0)": ['inproc://192.168.0.188/1007/3'], "('mean_agg-aggregate-8a3aa5d5a7c11071fd31b41c9dfb8b0c', 6)": ['inproc://192.168.0.188/1007/3'], "('mean_agg-aggregate-8a3aa5d5a7c11071fd31b41c9dfb8b0c', 9)": ['inproc://192.168.0.188/1007/3'], "('mean_agg-aggregate-8a3aa5d5a7c11071fd31b41c9dfb8b0c', 2)": ['inproc://192.168.0.188/1007/3'], "('mean_agg-aggregate-8a3aa5d5a7c11071fd31b41c9dfb8b0c', 3)": ['inproc://192.168.0.188/1007/3'], "('mean_agg-aggregate-8a3aa5d5a7c11071fd31b41c9dfb8b0c', 7)": ['inproc://192.168.0.188/1007/3'], "('mean_agg-aggregate-8a3aa5d5a7c11071fd31b41c9dfb8b0c', 8)": ['inproc://192.168.0.188/1007/3'], "('mean_agg-aggregate-8a3aa5d5a7c11071fd31b41c9dfb8b0c', 4)": ['inproc://192.168.0.188/1007/3'], "('mean_agg-aggregate-8a3aa5d5a7c11071fd31b41c9dfb8b0c', 5)": ['inproc://192.168.0.188/1007/3']} state: ['memory', 'memory', 'memory', 'memory', 'memory', 'memory', 'memo

array([ 2.18844347,  2.26968491,  2.59434695, ..., -1.45041756,
        0.59989791,  1.95754821])